# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/weather.csv")
df


,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Windspeed,Date
0,chokwe,MZ,-24.5333,32.9833,61.39,0,62,6.13,1627412320
1,port elizabeth,ZA,-33.9180,25.5701,62.89,0,82,16.11,1627411994
2,samarai,PG,-10.6167,150.6667,78.04,67,87,12.35,1627412100
3,punta arenas,CL,-53.1500,-70.9167,42.91,75,93,9.22,1627411957
4,atuona,PF,-9.8000,-139.0333,78.10,35,65,18.90,1627411966
...,...,...,...,...,...,...,...,...,...
563,cumana,VE,10.4635,-64.1775,86.09,13,76,10.96,1627412521
564,san patricio,MX,19.2167,-104.7000,87.73,42,61,5.59,1627412022
565,vitina,GR,41.4278,24.7646,73.35,0,50,3.89,1627412522
566,vangaindrano,MG,-23.3500,47.6000,62.42,95,87,6.98,1627412523


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

# grab maps
gmaps.configure(api_key=g_key)
# store lats/longs and humidity
Locations = df[["Lat", "Lng"]]
hum = df["Humidity"].astype(float)

In [4]:
ax = gmaps.figure(center = (46.0, -5.0), zoom_level=2)
max_intensity = np.max(hum)
# Create heat layer
heat = gmaps.heatmap_layer(Locations, weights = hum, max_intensity=100, point_radius=3, dissipating=False)
# Add heat layer and display map
ax.add_layer(heat)
ax

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
new_df = df.loc[(df["Windspeed"] <= 10 ) & (df["Cloudiness"] == 0) & \
(df["Max Temp"] > 70) & (df["Max Temp"] < 80)].dropna()

new_df.head(15)

,City,Country,Lat,Lng,Max Temp,Cloudiness,Humidity,Windspeed,Date
192,tiznit,MA,29.5833,-9.5000,74.37,0,62,6.51,1627412122
442,bardiyah,LY,31.7561,25.0865,79.56,0,64,8.32,1627412475
456,timizart,DZ,36.8000,4.2667,76.87,0,71,4.90,1627412481
495,lozova,UA,48.8894,36.3176,71.28,0,46,5.70,1627412495
505,karacabey,TR,40.2152,28.4038,75.13,0,64,7.45,1627412500
524,kushima,JP,31.4583,131.2333,73.76,0,88,2.91,1627412507
565,vitina,GR,41.4278,24.7646,73.35,0,50,3.89,1627412522


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = new_df.loc[:,("City", "Country", "Lat", "Lng")]
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
192,tiznit,MA,29.5833,-9.5000,
442,bardiyah,LY,31.7561,25.0865,
456,timizart,DZ,36.8000,4.2667,
495,lozova,UA,48.8894,36.3176,
505,karacabey,TR,40.2152,28.4038,


In [29]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel", 
          "radius" : 5000,
          "key" : g_key}


In [31]:
import pprint
import json

for index, row in hotel_df.iterrows():
    
    Lat=row["Lat"]
    Lng=row["Lng"]
    params["location"] = f"{Lat},{Lng}"
    name=row["City"]
    
    print(f"Retrieving Results for Index {index}: {name}.")
    response = requests.get(url, params=params).json()
    pprint.pprint(response)
    # extract results
    results = response['results']
    print(json.dumps(response, indent=4, sort_keys=True))    

Retrieving Results for Index 192: tiznit.
{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 29.6242257, 'lng': -9.5395357},
                           'viewport': {'northeast': {'lat': 29.65324385224401,
                                                      'lng': -9.529137628055702},
                                        'southwest': {'lat': 29.60446892018129,
                                                      'lng': -9.556088393051116}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'icon_background_color': '#7B9EB0',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet',
              'name': 'Ouijjane',
              'photos': [{'height': 1920,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/114799378576988832288">El '
             

{'html_attributions': [],
 'next_page_token': 'Aap_uECxw-mrkw4kK2jccs4QcWmVgR-Ux4bzP6C7YvAOd_04GMZhYQ0pJV0YQ4TmLS7DOk7QzWgfgMzhntWji1P7L2CYLAy_WOJ46TfBP8PqNRJWEaJZgqS7Ss_ccwVX_1cCT4hYVDra4S22RXoclcsZ__kcjf0-R3yLezp6TWwDMVk42zUqRb1O4-TYsWT1IVevwtMaHiTOZBLValUSOn_IRopeBeg_eRjrgLALM_j3mV4Ciw6U5kH2Qz_kuDnMEzoocB03qyX9v-y3htXIf3tAUdMAMsANf6MvyqkWCTCP7UxIWci4BLEpJOLAB-uk0uk5nxmqasnZNZ7edRyRK83Iic0LP1Nvvl6SXSvdgAF5AfCcSIdAHUOdSCG78Pf7JV8O09MUGt_F023WMgczXqKjP7CU4IBfgjkbjvO7orYJ1ujmPQWBM_vSSA',
 'results': [{'geometry': {'location': {'lat': 31.7579946, 'lng': 25.0792189},
                           'viewport': {'northeast': {'lat': 31.76531717929866,
                                                      'lng': 25.09696643632311},
                                        'southwest': {'lat': 31.75262405010896,
                                                      'lng': 25.06751454221766}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
     

{'html_attributions': [],
 'next_page_token': 'Aap_uECI85HLrlpZ15k7Wp1SvSDzXR2nR05KvTs7b3JoB9WjwEHQpP-muiXzww_yecgWE9uFrSyDiHvFlkKBr-hNxnGj34DAUKhvxiNw1iyyeS3Mp-uIYpRU71EWOkOFcn0US5k0nWtN-V_D3QTDnFNTNWMhzLpSdsEdJyg60hr77eSwyWtlNg8fsTra-X9hJ_w5mlbWvd-bWB5S5pM-psydD9dKONZAfI49MhQXqI51awvIQLalCOedu6Ei8yWMJSQ7gPxl8wJR10EpyYeRiERve8GXJUXDQ54CugjXS42KnDpc7rdaHTdYmr4GtoFy2LaMOzBvwwv29YqR7ALqPbTzHorPBW_TBfVZLRwZj064M3fbzFCbEAmDU0uNTqtBwxQPsH6NyRZ-9B0P6UsO9p-pCs_x6ZDu4wFkIvcqE-Up65MpNYxuNj50Mg',
 'results': [{'geometry': {'location': {'lat': 36.7940916, 'lng': 4.2449425},
                           'viewport': {'northeast': {'lat': 36.84573095175643,
                                                      'lng': 4.292840012472547},
                                        'southwest': {'lat': 36.74957002236297,
                                                      'lng': 4.180599842441615}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
      

{'html_attributions': [],
 'next_page_token': 'Aap_uEBW8le9Be2WukB4Kofr-RRPsKIPoTeajveadSJgvVeB2SrvrShMRwb0vpVt5Jmje8fP1-mtdhR6fgDuzTWA9u0AH_gZuIHp_L1eQZjRUx2MHoyEVsFXhdXtP2wW5fO0P2Ota3ceq81bvJfxqU7y8yqX7hLgxgAX7_vJa0LbCcyWob6SYaJL9BqtRRqctb-wXibPWfnylN3faVQJmUU76zdW7cSjTCGWgSWR9tnEeTXi7N89gEeBJppkzrxzeZgCruqrf5_CMsilp2Z_faU4T6w7--fJ83EeRDoJe-KsAv13CUEviPT9Fp_d-nqHprhb-DjJ3VaKVEYsdSl4W5tLl05lE7vcsuKZXE6AB9bHXO9du7TW0kixB4Ybq3wt59OSLKho3X9gc1PGDL5P1xcQrweahhFzW-QOdI0AVGaXPn5b7m5TuqDD6g',
 'results': [{'geometry': {'location': {'lat': 48.8902362, 'lng': 36.308001},
                           'viewport': {'northeast': {'lat': 48.92235824958278,
                                                      'lng': 36.37609717667612},
                                        'southwest': {'lat': 48.86654979231389,
                                                      'lng': 36.25556946981604}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
      

{'html_attributions': [],
 'next_page_token': 'Aap_uECo7RASDu7LJiIqwvb2zSxS_dX5D2cXehEHSr5WWvvHWOe9vxTqtnAW0NIQHV4qMlcdp5xU2HvvhZk635BMicIOzGmrjJzjW1CTayZBFMMXfHLm_kgVtG9yuH1lALp1jcK0OBeAXstWEhw6F7oIJpUmWNG9ezekoYqE8FtbOTMHGwlqNfD7FPO0Fm-6JjG5f1HZTLJJag7oR03Xi0EbMgmPc1FiKQzWL_EnljsuRYPr4ZPwsYKSBYqTrsJAI5X9G8fW670176dwo55MtTaKabRqnzw3mL_KSsA8tjLG0RdheXUnVo09VnwdDZMLEKV3TeBONbDpcMTA6OU7NXz_xLkY87HxhdY4_TjUMn-FvgbkqXTienDR8Gv_qpl-stahv9Xo4wtT7fktcZKYPAnGDSZXGV2JKydS0EZeYqSQU8CKkqxSIy6BzQ',
 'results': [{'geometry': {'location': {'lat': 40.216037, 'lng': 28.359},
                           'viewport': {'northeast': {'lat': 40.22923499575046,
                                                      'lng': 28.37520192925661},
                                        'southwest': {'lat': 40.19644404135572,
                                                      'lng': 28.34981092053588}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
          

{'html_attributions': [],
 'next_page_token': 'Aap_uEBlFr79yWfXJwCSbYts1zwAaH_UiYYzdcSiJxpRBbi1B5YuntpnSig0LK0cjt0sfV8mSFGfKjPDx6EWd49Vh2Em5hYBqfeoBWW3khLug5YrumkOm5Iqx7yx8iQci-GQb0nQdsulFm_oTRh4FUwHf9gRBcZGTmtlj7pB_FU6yZqKODcZ7_1J4Z6BE4TGsszlqS38j1WBWfjrOXWam13VytuBV0qkQJUtJv44GQSQKkW0GlyII5Q2Q3fxLd9S_e_8w2va04MvACYGHf-ueFuHubRZnG_TzByfWoBkxDohBnDAi0iaITy-7KYGJOhZglOvwR5kFULQDpeH0oF9j9CR_uPlP7Qf-WQLvmJ-nT2jKmsIEw9ryKkV3KFsr9gT9W8qtGPhaIuLbdTQT6RUlrzDo0yWq4C8BmG7WlvUuWtRQEc6cRK1pBjM3Q',
 'results': [{'geometry': {'location': {'lat': 31.4645293, 'lng': 131.2283553},
                           'viewport': {'northeast': {'lat': 31.63734504353898,
                                                      'lng': 131.425070724201},
                                        'southwest': {'lat': 31.35642543746468,
                                                      'lng': 131.1597404521337}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
     

{'html_attributions': [],
 'next_page_token': 'Aap_uECQ6SnA20YFY0di71Q2obXvBz-mZhtPqmYaoGHjRFaojsTxiUqWRXH_LPPUhsWedzRGzl_-sdZJaRV_Iflt1BbtZJOaNnZdCnjYkPh04UMQXgPivQNpqlMvU8lQ6jPIlZXLKCyRC6LoFaR2qiM75NUKg3xEfy1YJebHm5x9PgEJBR_oMAver2hyg_NDQFGYtDmZvMymcmV3sa6-cftNw9qbervAY13ZNfkEnTVb5u4vvwUDhBZ8gIHqMPKyoqbC-f7t9viaBo5I3_cn_PuVnPzl33loYYtMiZd8A-X8baqSXNr4tJDyjqaOm8d71IHyPmlKfhnfhw840lx_sWDLwiKIkqyir1ajIasAm-M9PIF6lMYoK8uu0De7UurAE2gJB-VF8YWtAFNjbwKDjwlrjm8hYfUlkGDot2nH4yrdArpfi5eaUh-UNQ',
 'results': [{'geometry': {'location': {'lat': 41.435631, 'lng': 24.7331761},
                           'viewport': {'northeast': {'lat': 41.43895804694434,
                                                      'lng': 24.73593396763704},
                                        'southwest': {'lat': 41.43425992464061,
                                                      'lng': 24.73049164898843}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
      

In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
ax.add_layer(marker_layer)
# Display figure
ax

Figure(layout=FigureLayout(height='420px'))